# Let's find number of incidents 

In [1]:
import requests

def get_data_from_uri(uri):
    # Set up the headers with the API key
    headers = {
    "Content-Type": "application/json"
    }

    # Send a GET request to the API endpoint
    response = requests.get(uri, headers=headers)

    # Check the status code of the response
    if response.status_code == 200:
        data = response.json()
        return data
        
    else:
        print(f"Request failed with status code {response.status_code}")
        print(response.text)
        return None

In [2]:
import pandas as pd

# Send HTTP requests
agency_data = get_data_from_uri("https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/GA?API_KEY=KPY34tj4bkbX8N0wgKtvtQncnfFdubWDYzZTekYR")

# If the HTTP request return data, then convert the json data to DataFrame
if agency_data:
    agency_df = pd.DataFrame(agency_data)
    print(agency_df.head(3)) # Printing the top 3 rows of the dataframe to view the values

         ori                      agency_name  agency_id state_name  \
0  GA0010000  Appling County Sheriff's Office       3468    Georgia   
1  GA0010100         Baxley Police Department       3469    Georgia   
2  GA0010300                           Graham      26899    Georgia   

  state_abbr   division_name region_name region_desc county_name  \
0         GA  South Atlantic       South  Region III     APPLING   
1         GA  South Atlantic       South  Region III     APPLING   
2         GA  South Atlantic       South  Region III     APPLING   

  agency_type_name  nibrs          nibrs_start_date   latitude   longitude  
0           County  False  2019-10-01T00:00:00.000Z  31.782509   -82.35828  
1             City   True  2018-12-01T00:00:00.000Z  31.739712  -82.290103  
2             City  False                      None       None        None  


In [3]:
# To get the zipcode we need latitude and longitude, so we need to find latitude and longitude

# We are iterating through rows using agency_df.iterrows() to get the latitude and longitude values from each rows
for index, row in agency_df.iterrows():
    print(row['latitude'], row['longitude'])
    break  # We are adding break statement here since we just want to look into the one row value and stop printing the data

31.782509 -82.35828


## Let's try to find zipcode from one of these values

In [4]:
# pip install arcgis. To install library
from arcgis.gis import GIS
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Point

# Reverse geocode find the address from the latitude and longitude

gis = GIS()

# X has the value of longitude and Y has the value of latitude
point = Point({"X": -84.30061, "Y":32.898376})

# reverse geocode to find the address from the latitude and longitude
result = reverse_geocode(point)
print("Result:", result)

# Result is in JSON form. Since we just want the zipcode. We will lookup from the address key and Postal key.
zip_code = result['address']['Postal']
print("Zipcode:", zip_code)

/userapp/workshop24/venv/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Result: {'address': {'Match_addr': 'Knight Trl, Thomaston, Georgia, 30286', 'LongLabel': 'Knight Trl, Thomaston, GA, 30286, USA', 'ShortLabel': 'Knight Trl', 'Addr_type': 'StreetName', 'Type': '', 'PlaceName': '', 'AddNum': '', 'Address': 'Knight Trl', 'Block': '', 'Sector': '', 'Neighborhood': '', 'District': '', 'City': 'Thomaston', 'MetroArea': '', 'Subregion': 'Upson County', 'Region': 'Georgia', 'RegionAbbr': 'GA', 'Territory': '', 'Postal': '30286', 'PostalExt': '', 'CntryName': 'United States', 'CountryCode': 'USA', 'X': -84.300535435154, 'Y': 32.898392718671, 'InputX': -84.30061, 'InputY': 32.898376}, 'location': {'x': -84.300535435154, 'y': 32.898392718671, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}}
Zipcode: 30286


## Let's create a function to get zipcode from the latitude and longitude



In [5]:
# Create a function that gets latitude and longitude and return the zipcode.
# This function will take latitude and longitude as argument, do the processing and return zip code.

def get_zipcode_from_latlng(latitude, longitude):
    gis = GIS()
    point = Point({"X": longitude, "Y":latitude})
    location = reverse_geocode(point)
    zipcode = location['address']['Postal']
    return zipcode

## We are getting error "Exception: Unable to complete operation.The operation was attempted on an empty geometry (Error Code: 400)" 

In the function 

zipcode = get_zipcode_from_latlng(latitude, longitude), it's throwing error.

That means while trying to get zipcode from latitude and longitude, some of the rows didn't have the value latitude and longitude. That's why it's throwing error.


Let's look at the latitude and longitude data if there are any empty values

In [14]:
for index, row in agency_df.iterrows():
    # get the latitude value from the latitude column
    latitude = row['latitude']
    print(latitude)
    # Get the longitude value from the longitude column
    longitude = row ['longitude']
    print(longitude)

31.782509
-82.35828
31.739712
-82.290103
None
None
None
None
31.296677
-82.878067
31.296677
-82.878067
31.340805
-83.04818
31.563327
-82.387859
31.563327
-82.387859
31.31958
-84.454881
31.31958
-84.454881
33.059504
-83.255452
33.082405
-83.228714
33.059504
-83.255452
33.08015
-83.22491
34.351922
-83.498441
33.992009
-83.712303
33.993984
-83.71819
33.992009
-83.712303
33.992009
-83.712303
34.240917
-84.838188
34.240917
-84.838188
34.368633
-84.93449
34.240917
-84.838188
34.240917
-84.838188
34.240917
-84.838188
34.240917
-84.838188
None
None
None
None
31.740775
-83.14719
31.740775
-83.14719
31.191711
-83.27352
31.274308
-83.231906
31.274308
-83.231906
31.274308
-83.231906
31.07462
-83.20011
32.831833
-83.64161
32.808844
-83.694193
32.841473
-83.660034
32.82903
-83.648766
32.435403
-83.331717
32.386444
-83.35458
32.435403
-83.331717
31.197334
-81.982978
31.197334
-81.982978
31.197334
-81.982978
30.822934
-83.581905
30.7866
-83.5581
30.822934
-83.581905
32.017969
-81.438543
32.017969
-81.

## Looking at the list of latitude and longitude output, we can see that we have some None values in the dataset.

Let's delete those values 

In [16]:
agency_df.shape

(662, 14)

In [18]:
# We are creating new dataframe that doesn't have None value in latitude and longitude.
# We are doing that by dropping rows  if the column latitude and longitude have nan values
agency_df_latlng = agency_df.dropna(subset=["latitude","longitude"])


In [19]:
agency_df_latlng.shape

(629, 14)

In [22]:
## Let's call that function to get zipcode value for all of the agency name
zipcodes = []

# We will iterate through the new dataframe where we deleted the rows where the values of column latitude  and longitude are None
for index, row in agency_df_latlng.iterrows():
    latitude = row['latitude']
    longitude = row ['longitude']
    zipcode = get_zipcode_from_latlng(latitude, longitude)
    zipcodes.append(zipcode)

print(len(zipcodes))

629


## Now let's append the zipcodes by creating a new column in the agency_df_latlng datafraem

In [17]:
## Let's work with new dataframe and add zipcode column to that dataframe 

In [24]:
# To add the column with datasets, we can just call df['new_column_name']=column_data
agency_df_latlng['zipcode'] = zipcodes

/tmp/ipykernel_1676174/1058039803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agency_df_latlng['zipcode'] = zipcodes


In [25]:
agency_df_latlng.shape

(629, 15)

In [26]:
agency_df_latlng.head()

,ori,agency_name,agency_id,state_name,state_abbr,division_name,region_name,region_desc,county_name,agency_type_name,nibrs,nibrs_start_date,latitude,longitude,zipcode
0,GA0010000,Appling County Sheriff's Office,3468,Georgia,GA,South Atlantic,South,Region III,APPLING,County,False,2019-10-01T00:00:00.000Z,31.782509,-82.35828,31513
1,GA0010100,Baxley Police Department,3469,Georgia,GA,South Atlantic,South,Region III,APPLING,City,True,2018-12-01T00:00:00.000Z,31.739712,-82.290103,31513
4,GA0020000,Atkinson County Sheriff's Office,3471,Georgia,GA,South Atlantic,South,Region III,ATKINSON,County,True,2019-10-01T00:00:00.000Z,31.296677,-82.878067,31642
5,GA0020100,Pearson Police Department,3472,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,None,31.296677,-82.878067,31642
6,GA0020200,Willacoochee Police Department,3473,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,None,31.340805,-83.04818,31650


In [27]:
# Save this dataframe to csv file

agency_df_latlng.to_csv('agency_with_zipcodes.csv', index=False)

In [12]:
# Now, let's see the number of unique zipcodes in the dataframe
agency_df_latlng = pd.read_csv('agency_with_zipcodes.csv')
agency_df_latlng.zipcode.nunique()

230

In [13]:
agency_df_latlng.ori.nunique()

629

In [14]:
agency_df_latlng.head()

,ori,agency_name,agency_id,state_name,state_abbr,division_name,region_name,region_desc,county_name,agency_type_name,nibrs,nibrs_start_date,latitude,longitude,zipcode
0,GA0010000,Appling County Sheriff's Office,3468,Georgia,GA,South Atlantic,South,Region III,APPLING,County,False,2019-10-01T00:00:00.000Z,31.782509,-82.358280,31513.0
1,GA0010100,Baxley Police Department,3469,Georgia,GA,South Atlantic,South,Region III,APPLING,City,True,2018-12-01T00:00:00.000Z,31.739712,-82.290103,31513.0
2,GA0020000,Atkinson County Sheriff's Office,3471,Georgia,GA,South Atlantic,South,Region III,ATKINSON,County,True,2019-10-01T00:00:00.000Z,31.296677,-82.878067,31642.0
3,GA0020100,Pearson Police Department,3472,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,NaN,31.296677,-82.878067,31642.0
4,GA0020200,Willacoochee Police Department,3473,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,NaN,31.340805,-83.048180,31650.0


## Make key value pair dictionary with zipcode as keys and ori as values for that keys

In [15]:
# Reset the index

agency_df_latlng = agency_df_latlng.reset_index(drop=True)

In [16]:
agency_df_latlng.head()

,ori,agency_name,agency_id,state_name,state_abbr,division_name,region_name,region_desc,county_name,agency_type_name,nibrs,nibrs_start_date,latitude,longitude,zipcode
0,GA0010000,Appling County Sheriff's Office,3468,Georgia,GA,South Atlantic,South,Region III,APPLING,County,False,2019-10-01T00:00:00.000Z,31.782509,-82.358280,31513.0
1,GA0010100,Baxley Police Department,3469,Georgia,GA,South Atlantic,South,Region III,APPLING,City,True,2018-12-01T00:00:00.000Z,31.739712,-82.290103,31513.0
2,GA0020000,Atkinson County Sheriff's Office,3471,Georgia,GA,South Atlantic,South,Region III,ATKINSON,County,True,2019-10-01T00:00:00.000Z,31.296677,-82.878067,31642.0
3,GA0020100,Pearson Police Department,3472,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,NaN,31.296677,-82.878067,31642.0
4,GA0020200,Willacoochee Police Department,3473,Georgia,GA,South Atlantic,South,Region III,ATKINSON,City,False,NaN,31.340805,-83.048180,31650.0


In [17]:
# Creating a dataframe with ori and zipcode value only

ori_zip_df = agency_df_latlng.loc[:,['ori', 'zipcode']]

In [18]:
# Printing the top 5 values of the ori_zip_df 

ori_zip_df.head()

,ori,zipcode
0,GA0010000,31513.0
1,GA0010100,31513.0
2,GA0020000,31642.0
3,GA0020100,31642.0
4,GA0020200,31650.0


In [19]:
# Create an empty dictionary

ori_zip_dict = {}

# Loop through the dataframe and get zipcode and ori value.
for index, row in agency_df_latlng.iterrows():
    key = row['zipcode']
    value = row['ori']
    # Add zipcode as key and value as list of ori. So, that we can see how may agencies is in each zipcode.
    ori_zip_dict.setdefault(key, []).append(value)



In [20]:
print(ori_zip_dict)

{31513.0: ['GA0010000', 'GA0010100'], 31642.0: ['GA0020000', 'GA0020100'], 31650.0: ['GA0020200'], 31510.0: ['GA0030000', 'GA0030100'], 39870.0: ['GA0040000', 'GA0040100'], nan: ['GA0050000'], 31061.0: ['GA0050100', 'GA0050400'], nan: ['GA0050300'], nan: ['GA0060000'], 30680.0: ['GA0070000', 'GA0070100', 'GA0070200', 'GA0070300'], 30121.0: ['GA0080000', 'GA0080100', 'GA0080300', 'GA0080500', 'GA0080600', 'GA0080700', 'GAGSP0300'], 30103.0: ['GA0080200'], nan: ['GA0090000'], nan: ['GA0090100'], 31639.0: ['GA0100000', 'GA0100100', 'GA0100200', 'GA0100300'], 31645.0: ['GA0100400'], 31201.0: ['GA0110000'], 31206.0: ['GA0110200'], 31204.0: ['GA0110400'], 31207.0: ['GA011099E'], 31014.0: ['GA0120000', 'GA0120100', 'GA0120200'], 31553.0: ['GA0130000', 'GA0130100', 'GA0130200'], 31643.0: ['GA0140000', 'GA0140100', 'GA0140200'], 31314.0: ['GA0150000', 'GA0150100', 'GA0150200'], 30458.0: ['GA0160000', 'GA0160100', 'GAGSP4500'], 30461.0: ['GA0160200', 'GA0160300', 'GA0160400'], 30830.0: ['GA01700

## Find how many burgarly happens in each zipcode

In [21]:
import pandas as pd

arrest_df = pd.read_csv('arrests.csv')
arrest_df.head()

,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,Drunkenness,...,Rape,Robbery,Simple Assault,"Stolen Property: Buying, Receiving, Possessing",Suspicion,Vagrancy,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",ori
0,2011.0,7.0,110.0,0.0,13.0,0.0,38.0,97.0,51.0,5.0,...,0.0,0.0,14.0,1.0,0.0,0.0,5.0,0.0,0.0,GA0010000
1,2012.0,10.0,66.0,3.0,12.0,0.0,17.0,66.0,36.0,3.0,...,0.0,0.0,17.0,3.0,0.0,0.0,1.0,5.0,9.0,GA0010000
2,2013.0,3.0,31.0,0.0,16.0,0.0,16.0,19.0,39.0,2.0,...,1.0,0.0,5.0,0.0,0.0,0.0,5.0,1.0,3.0,GA0010000
3,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GA0010000
4,2016.0,6.0,52.0,0.0,33.0,0.0,18.0,21.0,64.0,0.0,...,1.0,0.0,15.0,0.0,1.0,0.0,6.0,5.0,7.0,GA0010000


## How many total Burglary happened in zipcode 31513 

In [22]:
arrest_31513 = arrest_df.loc[(arrest_df['ori'] == 'GA0010000') | (arrest_df['ori'] == 'GA0010100')]

In [23]:
arrest_31513

,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,Drunkenness,...,Rape,Robbery,Simple Assault,"Stolen Property: Buying, Receiving, Possessing",Suspicion,Vagrancy,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",ori
0,2011.0,7.0,110.0,0.0,13.0,0.0,38.0,97.0,51.0,5.0,...,0.0,0.0,14.0,1.0,0.0,0.0,5.0,0.0,0.0,GA0010000
1,2012.0,10.0,66.0,3.0,12.0,0.0,17.0,66.0,36.0,3.0,...,0.0,0.0,17.0,3.0,0.0,0.0,1.0,5.0,9.0,GA0010000
2,2013.0,3.0,31.0,0.0,16.0,0.0,16.0,19.0,39.0,2.0,...,1.0,0.0,5.0,0.0,0.0,0.0,5.0,1.0,3.0,GA0010000
3,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GA0010000
4,2016.0,6.0,52.0,0.0,33.0,0.0,18.0,21.0,64.0,0.0,...,1.0,0.0,15.0,0.0,1.0,0.0,6.0,5.0,7.0,GA0010000
5,2017.0,8.0,89.0,2.0,14.0,0.0,11.0,24.0,64.0,3.0,...,0.0,7.0,14.0,11.0,0.0,0.0,4.0,2.0,8.0,GA0010000
6,2018.0,14.0,160.0,0.0,16.0,0.0,32.0,18.0,117.0,3.0,...,0.0,0.0,14.0,4.0,0.0,0.0,4.0,11.0,9.0,GA0010000
7,2019.0,6.0,148.0,1.0,5.0,0.0,16.0,27.0,124.0,5.0,...,0.0,0.0,15.0,5.0,0.0,0.0,3.0,1.0,5.0,GA0010000
8,2020.0,1.0,3.0,0.0,1.0,0.0,5.0,5.0,16.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,GA0010000
9,2010.0,2.0,0.0,0.0,0.0,0.0,4.0,15.0,2.0,0.0,...,0.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,GA0010100


In [24]:
arrest_31513['Burglary'].sum()

158.0



## Let's find out how many burglary happened in zipcode 31513 

In [25]:
arrest_31513.groupby('data_year')['Burglary'].sum()

data_year
2010.0     0.0
2011.0    14.0
2012.0    15.0
2013.0    23.0
2014.0     9.0
2015.0     0.0
2016.0    34.0
2017.0    17.0
2018.0    26.0
2019.0    12.0
2020.0     3.0
2021.0     3.0
2022.0     2.0
Name: Burglary, dtype: float64

In [26]:
arrest_df.shape

(5664, 33)

In [27]:
#arrest_df
#ori_zip_df


arrest_zip_df = pd.merge(arrest_df, ori_zip_df, on='ori')
arrest_zip_df.head()

,data_year,Aggravated Assault,All Other Offenses (Except Traffic),Arson,Burglary,Curfew and Loitering Law Violations,Disorderly Conduct,Driving Under the Influence,Drug Abuse Violations - Grand Total,Drunkenness,...,Robbery,Simple Assault,"Stolen Property: Buying, Receiving, Possessing",Suspicion,Vagrancy,Vandalism,"Weapons: Carrying, Possessing, Etc.","Sex Offenses (Except Rape, and Prostitution and Commercialized Vice)",ori,zipcode
0,2011.0,7.0,110.0,0.0,13.0,0.0,38.0,97.0,51.0,5.0,...,0.0,14.0,1.0,0.0,0.0,5.0,0.0,0.0,GA0010000,31513.0
1,2012.0,10.0,66.0,3.0,12.0,0.0,17.0,66.0,36.0,3.0,...,0.0,17.0,3.0,0.0,0.0,1.0,5.0,9.0,GA0010000,31513.0
2,2013.0,3.0,31.0,0.0,16.0,0.0,16.0,19.0,39.0,2.0,...,0.0,5.0,0.0,0.0,0.0,5.0,1.0,3.0,GA0010000,31513.0
3,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,GA0010000,31513.0
4,2016.0,6.0,52.0,0.0,33.0,0.0,18.0,21.0,64.0,0.0,...,0.0,15.0,0.0,1.0,0.0,6.0,5.0,7.0,GA0010000,31513.0


In [33]:
## Find the sum of the burglary incident that happened in each zipcode

burglary = arrest_zip_df.groupby('zipcode')['Burglary'].sum()

In [34]:
burglary = burglary[1:]
burglary_df = pd.DataFrame(burglary)
burglary_df = burglary_df.reset_index()
burglary_df = burglary_df.astype(int)
burglary_df.head()

/tmp/ipykernel_1680016/2517222601.py:1: FutureWarning:

The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead



,zipcode,Burglary
0,30002,27
1,30004,38
2,30012,656
3,30014,326
4,30016,572


In [37]:
import plotly.express as px


from urllib.request import urlopen
import json


with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ga_georgia_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

# Create choropleth map using Plotly Express
fig = px.choropleth(burglary_df,  # Input data
                    geojson=zipcodes,
                    locations='zipcode', # Specify location column
                    #locationmode='USA-states',  # Specify location mode
                    scope="usa",
                    color='Burglary',  # Specify color data
                    color_continuous_scale='Viridis',  # Choose color scale
                    labels={'Burglary': 'Burglary'},  # Customize labels
                    title='Choropleth Map'  # Set title
                    )

# Update layout
#fig.update_geos(showframe=False,  # Hide frame
                #projection_type='natural earth')  # Specify projection
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
# Save the map
fig.write_image("fig1.png")
